Sodelujoči

    Tomaž Štrus
    Anže Košir
    Jan Harej

O projektu: Iz platforme Steam, namenjenu predvsem za distrubicijo video iger, bomo zbrali podatke igrah več ljudi, ter analizirali stvari kot so npr. koliko iger, ki jih kupijo dejansko preigrajo, kolika časa bo nekdo zapravil za neko igro, kakšno igro bo kupil naslednjo, glede na svoje igre ipd.

Podatki: Uporabili bomo API, ki ga ponuja Steam: https://developer.valvesoftware.com/wiki/Steam_Web_API#JSON Tu lahko dobimo podatko o čisto vsemu kar potrebujemo npr. o uporabniku, njegove igre, prijatelji, koliko ur je igral neko igro, ... Dobili bomo podatke za npr. 100.000+ uporabnikov (prijatelji naših prijateljev, in njihov prijateljev itd.)


API KEY: 5F5DD2FA8A6C8646FCFE265C07BB90E5

In [1]:
#install libraries on windows
'''
!python -m pip install numpy
!python -m pip install matplotlib
!python -m pip install requests
'''

'\n!python -m pip install numpy\n!python -m pip install matplotlib\n!python -m pip install requests\n'

In [2]:
import requests
import json
import numpy
import datetime
import os
from inspect import getouterframes, currentframe

In [3]:
api_key = "5F5DD2FA8A6C8646FCFE265C07BB90E5"
global_player_ids = set()
my_id="76561198101569818"

In [4]:
def get_player_summary (id):
    try:
        data = requests.get( "http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key="+api_key+"&steamids="+id).json()
    except:
        return None
    return data
    

In [5]:
get_player_summary("76561198202423736")

{'response': {'players': [{'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb.jpg',
    'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb_full.jpg',
    'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/fe/fef49e7fa7e1997310d705b2a6158ff8dc1cdfeb_medium.jpg',
    'communityvisibilitystate': 3,
    'lastlogoff': 1426344031,
    'personaname': 'legendofseoul',
    'personastate': 0,
    'personastateflags': 0,
    'primaryclanid': '103582791429521408',
    'profileurl': 'http://steamcommunity.com/profiles/76561198202423736/',
    'steamid': '76561198202423736',
    'timecreated': 1419608805}]}}

In [6]:
def get_player_friends(id):
    try:
        data = requests.get( "http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key="+api_key+"&steamid="+id+"&relationship=friend").json()
    except:
        return None
    return data

In [7]:
get_player_friends(my_id)

In [24]:
def fill_player_ids_rec(id, num = 1000, already_called = set()):
    global global_player_ids
    already_called.add(id)
    print("recursive call of id: " + id + "  len: " + str(len(global_player_ids)))
    try:
        if get_player_summary(id)["response"]["players"][0]["profilestate"] != 1:
            print("not public")
            return None
    except KeyError:
        print("not public key error")
        return None
    try:
        data = get_player_friends(id)["friendslist"]["friends"]
    except KeyError:
        print("no friends")
        return None
    tmp_id = set()
    for friend in data:
        if len(global_player_ids) >= num:
            print("enough friends")
            break
        tmp_id.add(friend["steamid"])
        global_player_ids.add(friend["steamid"])
        #print("len: "+ str(len(global_player_ids)))
        
    print("added my friends" + "  len: " + str(len(global_player_ids)))
    
    for playerid in tmp_id:
        if len(global_player_ids) >= num:
            break
        try:
            data = get_player_friends(playerid)["friendslist"]["friends"]
        except KeyError:
            print("no friends")
        for fof in data:
            if len(global_player_ids) >= num:
                print("enough friends")
                break
            global_player_ids.add(fof["steamid"])
        #print("added friends of friend")
            
    print("added friends of friends" + "  len: " + str(len(global_player_ids)))
    
    for playerid in tmp_id:
        if len(global_player_ids) >= num:
            print("enough friends")
            break
        if playerid not in already_called:
            fill_player_ids_rec(playerid, num, already_called)
    
    

In [ ]:
global_player_ids = set()
fill_player_ids_rec(my_id, 100000)
global_player_ids
print(len(global_player_ids))

recursive call of id: 76561198101569818  len: 0
added my friends  len: 41
no friends
no friends
